In [2]:
# Load MedCPT

import numpy
import json
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch

model = AutoModel.from_pretrained("ncbi/MedCPT-Article-Encoder")
tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Article-Encoder")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/Users/vikram/env/graph_ml/lib/python3.9/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [3]:
def compute_embeddings(data, model, tokenizer):
    with torch.no_grad():
        encoded = tokenizer(
            data,
            truncation=True,
            padding=True,
            return_tensors='pt',
            max_length=512,
        )
        embeds = model(**encoded).last_hidden_state[:, 0, :]
    return embeds.numpy()

In [6]:
metadata_path = "../../oc_mini/metadata/oc_mini_node_metadata.csv"
metadata_df = pd.read_csv(metadata_path)
metadata_df.head()

,id,doi,title,abstract
0,128,10.1101/2021.05.10.443415,Improved protein contact prediction using dime...,AbstractDeep residual learning has shown great...
1,163,10.1101/2021.05.07.443114,Following the Trail of One Million Genomes: Fo...,AbstractSevere acute respiratory syndrome coro...
2,200,10.1101/2021.05.11.443555,Mechanism of molnupiravir-induced SARS-CoV-2 m...,Molnupiravir is an orally available antiviral ...
3,941,10.3390/ijms20020449,Bactericidal and Cytotoxic Properties of Silve...,Silver nanoparticles (AgNPs) can be synthesize...
4,1141,10.3390/ijms20040865,Silver Nanoparticles: Synthesis and Applicatio...,"Over the past few decades, metal nanoparticles..."


In [7]:
get_dict_from_df = lambda df: dict(zip(df['id'], df['title'] + ' ' + df['abstract']))

In [9]:
metadata_dict = get_dict_from_df(metadata_df)

In [10]:
def embed_from_id(id):
    return compute_embeddings(metadata_dict[id], model, tokenizer)

embed_from_id(128)

array([[ 2.14536458e-01,  3.00464213e-01,  2.43838429e-01,
         1.20291516e-01,  1.76745281e-02, -2.72362351e-01,
        -1.78717792e-01,  3.30635756e-01,  4.48155493e-01,
         4.76780534e-02,  6.32259846e-02,  7.85025060e-02,
         2.53432184e-01,  3.56028713e-02, -3.09725821e-01,
         2.85808966e-02, -1.14464533e+00, -3.59703779e-01,
         1.31689146e-01,  1.04904950e-01,  1.46144226e-01,
         4.75415498e-01, -3.67179543e-01, -3.97671908e-02,
        -1.29207581e-01,  9.81897339e-02, -2.66846716e-01,
        -3.84048253e-01, -1.33608341e-01,  7.21688122e-02,
         7.98781887e-02,  3.65626842e-01, -4.48953658e-01,
         5.52070081e-01, -8.78719687e-02,  1.15545340e-01,
         2.27816217e-02, -1.53949082e-01,  6.58397794e-01,
        -4.50299308e-02,  4.11785424e-01, -9.27044004e-02,
        -2.68816128e-02,  2.81053543e-01,  2.09516436e-01,
         1.50406688e-01, -8.32635239e-02, -1.35959461e-01,
         3.01522106e-01, -1.88459247e-01, -2.13569671e-0

In [13]:
import json

oc_paris_path = "../../oc_mini/clustering/hierarchical/oc_mini_paris.json"

with open(oc_paris_path, 'r') as f:
    oc_paris = json.load(f)

print(oc_paris)

{'algorithm': 'Paris', 'num_nodes': 14384, 'num_edges': 111873, 'hierarchy': {'id': 28766, 'type': 'cluster', 'distance': 2129.492497541242, 'count': 14384, 'children': [{'id': 28506, 'type': 'cluster', 'distance': 0.10218242328541843, 'count': 1267, 'children': [{'id': 28462, 'type': 'cluster', 'distance': 0.09025191585292336, 'count': 892, 'children': [{'id': 28386, 'type': 'cluster', 'distance': 0.07127057283113912, 'count': 567, 'children': [{'id': 28347, 'type': 'cluster', 'distance': 0.06481369591840531, 'count': 369, 'children': [{'id': 28245, 'type': 'cluster', 'distance': 0.05311330838446649, 'count': 238, 'children': [{'id': 28098, 'type': 'cluster', 'distance': 0.03986616281415472, 'count': 120, 'children': [{'id': 27888, 'type': 'cluster', 'distance': 0.029214179102897447, 'count': 87, 'children': [{'id': 26580, 'type': 'cluster', 'distance': 0.008810213211703565, 'count': 25, 'children': [{'id': 24891, 'type': 'cluster', 'distance': 0.0036469923331691324, 'count': 12, 'chi

In [17]:
from sklearn.metrics import silhouette_score, silhouette_samples
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

def get_clusters_at_distance(node, threshold_distance):
    """Extract flat clusters at a given distance threshold"""
    if node['type'] == 'leaf':
        return {node['name']: node['name']}
    
    if node['distance'] <= threshold_distance:
        # This node becomes a cluster
        leaves = get_all_leaves(node)
        cluster_id = node['id']
        return {leaf: cluster_id for leaf in leaves}
    
    # Recurse
    result = {}
    for child in node['children']:
        result.update(get_clusters_at_distance(child, threshold_distance))
    return result

def get_all_leaves(node):
    """Get all leaf names under a node"""
    if node['type'] == 'leaf':
        return [node['name']]
    leaves = []
    for child in node['children']:
        leaves.extend(get_all_leaves(child))
    return leaves

# Get embeddings for all leaves with progress bar
def compute_embeddings_for_leaves(hierarchy):
    leaves = get_all_leaves(hierarchy['hierarchy'])
    embeddings = {}
    for leaf in tqdm(leaves, desc="Computing embeddings"):
        embeddings[leaf] = embed_from_id(int(leaf))
    return embeddings

print("Computing embeddings for all leaves...")
embeddings = compute_embeddings_for_leaves(oc_paris)

# Test different distance thresholds
distances = [0.01, 0.05, 0.1, 0.5, 1.0, 5.0]
silhouette_scores = []

print("\nComputing silhouette scores at different thresholds...")
for dist in tqdm(distances, desc="Distance thresholds"):
    clusters = get_clusters_at_distance(oc_paris['hierarchy'], dist)
    
    # Create embedding matrix and label array
    leaf_ids = list(clusters.keys())
    X = np.array([embeddings[lid] for lid in leaf_ids])
    labels = np.array([clusters[lid] for lid in leaf_ids])
    
    # Only compute if we have more than 1 cluster
    n_clusters = len(set(labels))
    if n_clusters > 1 and n_clusters < len(leaf_ids):
        score = silhouette_score(X, labels)
        silhouette_scores.append(score)
        tqdm.write(f"Distance {dist}: {n_clusters} clusters, Silhouette: {score:.4f}")
    else:
        silhouette_scores.append(None)
        tqdm.write(f"Distance {dist}: {n_clusters} clusters, Silhouette: N/A")

# Plot
plt.figure(figsize=(10, 6))
valid_distances = [d for d, s in zip(distances, silhouette_scores) if s is not None]
valid_scores = [s for s in silhouette_scores if s is not None]
plt.plot(valid_distances, valid_scores, 'o-')
plt.xlabel('Distance Threshold')
plt.ylabel('Silhouette Score')
plt.title('Cluster Cohesiveness at Different Hierarchical Levels')
plt.grid(True)
plt.show()

Computing embeddings for all leaves...


Computing embeddings:  42%|██████▊         | 6081/14384 [35:27<48:24,  2.86it/s]


KeyboardInterrupt: 